In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master('local')
         .getOrCreate())
sc = spark.sparkContext

In [45]:
# Schema: https://github.com/wikimedia/mediawiki-event-schemas/blob/master/jsonschema/mediawiki/recentchange/1.0.0.json
# funcionou
import json
jsonData = """{
"$schema": "/mediawiki/recentchange/1.0.0",
"meta": {
    "uri": "https://de.wikipedia.org/wiki/Kategorie:Basketballtrainer_(Spanien)",
    "request_id": "5e8ab0ca-7cfc-4651-a47d-ce03baae3fbf",
    "id": "77e7b0b1-fd50-4e00-a785-f3f97227e1b8",
    "dt": "2020-06-27T10:26:39Z",
    "domain": "de.wikipedia.org",
    "stream": "mediawiki.recentchange",
    "topic": "eqiad.mediawiki.recentchange",
    "partition": 0,
    "offset": 2521360710
},
"id": 291966492,
"type": "categorize",
"namespace": 14,
"title": "Kategorie:Basketballtrainer (Spanien)",
"comment": "[[:Žan Tabak]] zur Kategorie hinzugefügt",
"timestamp": 1593253599,
"user": "Zamekrizeni",
"bot": false,
"server_url": "https://de.wikipedia.org",
"server_name": "de.wikipedia.org",
"server_script_path": "/w",
"wiki": "dewiki",
"parsedcomment": "<a href=\"/wiki/%C5%BDan_Tabak\" title=\"Žan Tabak\">Žan Tabak</a> zur Kategorie hinzugefügt"
}"""

In [67]:
# não funcionou
import json
jsonData = """{"$schema": "/mediawiki/recentchange/1.0.0", "meta": {"uri": "https://www.wikidata.org/wiki/Q95695404", "request_id": "378e375a-f30c-4532-a701-859402ab5286", "id": "7f443752-6d69-4946-b387-dc70719e89f9", "dt": "2020-06-27T09:44:17Z", "domain": "www.wikidata.org", "stream": "mediawiki.recentchange", "topic": "eqiad.mediawiki.recentchange", "partition": 0, "offset": 2521264632}, "id": 1257203778, "type": "edit", "namespace": 0, "title": "Q95695404", "comment": "/* wbsetreference-add:2| */ [[Property:P214]]: 25346950, [[:toollabs:quickstatements/#/batch/37736|batch #37736]]", "timestamp": 1593251057, "user": "Epìdosis", "bot": false, "minor": false, "patrolled": true, "length": {"old": 2277, "new": 3352}, "revision": {"old": 1217490529, "new": 1217905713}, "server_url": "https://www.wikidata.org", "server_name": "www.wikidata.org", "server_script_path": "/w", "wiki": "wikidatawiki", "parsedcomment": "\u200e<span dir=auto><span class=autocomment>Aggiunto riferimento a un\"asserzione: </span></span> <a href=/wiki/Property:P214 title=Property:P214>Property:P214</a>: 25346950, <a href=https://tools.wmflabs.org/quickstatements/#.2Fbatch.2F37736 class=extiw title=toollabs:quickstatements/>batch #37736</a>"}"""

In [68]:
!pwd

/home/jovyan/work/notebooks


In [69]:
df = spark.read.json(sc.parallelize([jsonData]), multiLine=True)
#df = spark.read.json("/home/jovyan/work/notebooks/data/wiki-changes-sample.json", multiLine=True)

In [70]:
df.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|{"$schema": "/med...|
+--------------------+



In [71]:
print(df)

DataFrame[_corrupt_record: string]


In [43]:
df.schema.json()

'{"fields":[{"metadata":{},"name":"$schema","nullable":true,"type":"string"},{"metadata":{},"name":"bot","nullable":true,"type":"boolean"},{"metadata":{},"name":"comment","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"long"},{"metadata":{},"name":"meta","nullable":true,"type":{"fields":[{"metadata":{},"name":"domain","nullable":true,"type":"string"},{"metadata":{},"name":"dt","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"offset","nullable":true,"type":"long"},{"metadata":{},"name":"partition","nullable":true,"type":"long"},{"metadata":{},"name":"request_id","nullable":true,"type":"string"},{"metadata":{},"name":"stream","nullable":true,"type":"string"},{"metadata":{},"name":"topic","nullable":true,"type":"string"},{"metadata":{},"name":"uri","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"namespace","nullable":true,"type":"long"},{"metadata":{},"name":"p

In [44]:
df.schema.simpleString()

'struct<$schema:string,bot:boolean,comment:string,id:bigint,meta:struct<domain:string,dt:string,id:string,offset:bigint,partition:bigint,request_id:string,stream:string,topic:string,uri:string>,namespace:bigint,parsedcomment:string,server_name:string,server_script_path:string,server_url:string,timestamp:bigint,title:string,type:string,user:string,wiki:string>'

In [25]:
import json
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType

new_schema = StructType.fromJson(json.loads(df.schema.json()))

In [26]:
print(new_schema)

StructType(List(StructField(_corrupt_record,StringType,true)))


In [36]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType

schema = StructType([
    StructField("$schema", StringType(), True),
    StructField("meta", 
                StructType([
                    StructField("uri", StringType(), True),
                    StructField("request_id", StringType(), True),
                    StructField("id", StringType(), True),
                    StructField("dt", TimestampType(), True),
                    StructField("domain", StringType(), True),
                    StructField("stream", StringType(), True),
                    StructField("topic", StringType(), True),
                    StructField("partition", StringType(), True),
                    StructField("offset", StringType(), True)
                ])
               ),
    StructField("id", StringType(), True),
    StructField("type", StringType(), True),
    StructField("namespace", StringType(), True),
    StructField("title", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("user", StringType(), True),
    StructField("bot", StringType(), True),
    StructField("server_url", StringType(), True),
    StructField("server_name", StringType(), True),
    StructField("server_script_path", StringType(), True),
    StructField("wiki", StringType(), True),
    StructField("parsedcomment", StringType(), True)
])

In [37]:
df1 = spark.createDataFrame(
    spark.sparkContext.parallelize(jsonData)
    ,schema
)

In [38]:
df1.show()

Py4JJavaError: An error occurred while calling o248.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/spark/python/pyspark/sql/session.py", line 730, in prepare
    verify_func(obj)
  File "/usr/local/spark/python/pyspark/sql/types.

In [20]:
jsonData1 = """{
  "type" : "struct",
  "fields" : [ {
        "name" : "$schema",
        "type" : "string",
        "nullable" : true,
        "metadata" : {}
      },
      {
        "name" : "meta",
        "type" : {
          "type" : "struct",
          "fields" : [
            {
              "name": "uri",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "request_id",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "id",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "dt",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "domain",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "stream",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "topic",
              "type": "string",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "partition",
              "type": "integer",
              "nullable": true,
              "metadata": {}
            },
            {
              "name": "offset",
              "type": "integer",
              "nullable": true,
              "metadata": {}
            }
          ]
        },
        "nullable" : true,
        "metadata" : {}
      },
      {
      "name": "id",
      "type": "integer",
      "nullable": true,
      "metadata": {}
      },
      {
        "name": "type",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "namespace",
        "type": "integer",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "title",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "comment",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "timestamp",
        "type": "integer",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "user",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "bot",
        "type": "boolean",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "server_url",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "server_name",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "server_script_path",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "wiki",
        "type": "string",
        "nullable": true,
        "metadata": {}
      },
      {
        "name": "parsedcomment",
        "type": "string",
        "nullable": true,
        "metadata": {}
      }
  ]
}
"""

In [27]:
schema_dict = json.loads(jsonData1)

In [28]:
from pyspark.sql.types import StructType
schema = StructType.fromJson(schema_dict)

In [29]:
print(schema)

StructType(List(StructField($schema,StringType,true),StructField(meta,StructType(List(StructField(uri,StringType,true),StructField(request_id,StringType,true),StructField(id,StringType,true),StructField(dt,StringType,true),StructField(domain,StringType,true),StructField(stream,StringType,true),StructField(topic,StringType,true),StructField(partition,IntegerType,true),StructField(offset,IntegerType,true))),true),StructField(id,IntegerType,true),StructField(type,StringType,true),StructField(namespace,IntegerType,true),StructField(title,StringType,true),StructField(comment,StringType,true),StructField(timestamp,IntegerType,true),StructField(user,StringType,true),StructField(bot,BooleanType,true),StructField(server_url,StringType,true),StructField(server_name,StringType,true),StructField(server_script_path,StringType,true),StructField(wiki,StringType,true),StructField(parsedcomment,StringType,true)))


In [30]:
df1 = spark.createDataFrame(
    spark.sparkContext.parallelize([jsonData])
    ,schema
)

In [31]:
df1.show()

Py4JJavaError: An error occurred while calling o232.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/spark/python/pyspark/sql/session.py", line 730, in prepare
    verify_func(obj)
  File "/usr/local/spark/python/pyspark/sql/types.